# Using R to analyze racial disparities in SF traffic stops

To start getting our hands dirty with applying our `R` skills, we're going to explore San Francisco's traffic stops. Our goals today are twofold:
1. To learn how to use `R` to deal with data and to answer policy questions. In addition to data-munging, we'll be computing descriptive statistics, plotting and even doing some modeling!
2. To get in the data science mindset: learning how to answer questions about social policies using data. In our case, we'll investigate whether there is racial discrimination in SF's policing practices, and we'll evaluate the impacts of certain policy changes like the legalization of marijuana.

## Getting started 

First, let's load the necessary libraries and data that will allow us to begin our investigation!

In [0]:
# Some initial setup
options(digits = 3)
library(tidyverse)
library(lubridate)
theme_set(theme_bw())
getwd()

# Read the data
stops <- read_rds("data/sf_stop_data.rds")
pop_2015 <- read_rds("data/sf_pop_2015.rds")

### Covering the basics

The core of `R` is the dataframe. We've given you one to start with, in the form
of `stops`. Think of dataframes like a spreadsheet: they have rows and columns.
Usually, rows are a "datapoint": in `stops`, each row corresponds to a single
stop from San Francisco. The columns are the "variables": again, in `stops`,
these are the things we know about the stop, like where the stop happened, the
age of the driver, whether an arrest was made, and so on.

We can take a peak simply by typing `stops` into an R chunk:

In [0]:
head(stops)

### Functions

Functions are a way to "do something" to an input. So `f(a)=b` takes a number `a`, and applies `f()`, and gets the output `b`. In programming, we also have functions! Most of the functions we'll use allow us to manipulate our dataframe as the input. 

The function `head(stops)` gave us the first 6 rows of our data frame. If we want to find the number of rows in our dataframe, we'd use the function `nrow()`, which takes a dataframe (like `stops`) as an input, and then outputs an integer (the number of rows in `stops`).

In [0]:
nrow(stops)

### Your turn

To find the number of columns, we (unsurprisingly) use `ncol`. Try it!

In [0]:
# Find the number of columns in `stops`


To figure out what the names of our columns are, we can use `colnames()`.

In [0]:
# Find the column names in `stops`


**Pro-tip:** If you're ever confused about a function and want to know more about it, what it does, how to use it, etc., every function has "documentation" to help! To know more about the `head()` function, simply run a code chunk with `?head`. It provides way more information than you might want or need -- but if you scroll down to the "Examples" section, those usually help!

## Exercise 1: Stop dates

**For this first exercise, let's get a better sense of what time range our `stops` data covers. To do this, we'll be dealing with the `date` column in our dataframe.**

**There are two ways to extract dates from our `stops` dataframe: `stops$date` and `pull(stops, date)`. Basically,`$` and `pull()` do the same thing.**

  1.  **Check `length(stops$date)` (or, equivalently, `length(pull(stops, date))`). Since this vector is too long to print, try instead _sampling_ a few dates from it using `sample()`, just like we did earlier today!**

  2.  **What date range does our dataset cover? (Hint: Try exploring the `min()` and `max()` functions, or the `range()` function!)**

In [0]:
## EXERCISE 1: YOUR CODE HERE


### New syntax 

Take a look at the two versions of the code below. Notice that they do the same thing. The "pipe" symbol -- `%>%` -- allows us to string functions together vertically, instead of nesting them.

In [0]:
# Confirm that these give the same answer:

# Method 1: nested
range(pull(stops, date))

# Method 2: multi-line
stops %>% 
    pull(date) %>% 
    range()


**tidyverse tip**: The second method uses a funky symbol, `%>%` called the "pipe", which is the crux of the tidyverse. The pipe helps to keep our code clean. It allows us to read top-down rather than inside-out (which is what method 1 above requires of us). Each line simply applies to the result of the previous line:
* We start with `stops`,
* then we apply `pull(date)` to the above (stops), getting us a list of dates,
* then we apply `range()` to the above (a list of dates).

More formally, the pipe operator
just places the previous item into the first argument of the function. So,
`x %>% f()` is simply `f(x)`. While in a one-function call the pipe might feel
silly and unnecessary, it's going to become _really_ helpful once we start
wanting to do multiple transformations to our data. 

## Preparing our data

For some of our analysis, we'll want to focus on the most recent full year: 2015.

To do this we'll want to use the _year_ of each stop, but _year_ isn't currently a column in our dataset. Let's add it!

**tidyverse function: `mutate()`**

We can use the `mutate()` function to fix add a `yr` column to `stops`.
The `mutate()` function adds new columns to a dataframe based on old columns.
The basic setup is `mutate(DATA, NEW_COL = FUN(OLD_COL))` where 
* `DATA` is our
dataframe, 
* `NEW_COL` is the name of the new column we want, and 
* `FUN` is the function we apply to the old column, `OLD_COL`, to get it.

### You try!

In the space below:

1. use the `year()` and `mutate()` functions to add a new column called `yr` to our `stops` dataframe, and
2. use the assignment operator `<-` (it's like = in `R`) to create a new variable, `stops_w_yr`.

In [0]:
# Add a yr column to `stops`


**Recall:** When we write code chunks and _don't_ save our result using `<-`, that result does not overwrite or in any way change the data. To change the data, we need to use the process above, creating a new variable, or we could overwrite the original dataframe (`stops <- stops %>% ...` -- but be careful, because you could accidentally overwrite the dataframe with something you didn't expect!)

Now, we can investigate this new `yr` column in a few ways. 
1. We can check it's acutally there by looking at `colnames(stops_w_yr)`.
2. We can compute the range of years using `range(stops_w_yr$yr)`.
3. We can count the number of stops per year: `stops_w_yr %>% count(yr)`. 

### You try

Play around with these! Make sure to try the last one.

In [0]:
# Investigate your new `yr` column. 
# Make sure to try counting the number of stops per year!


Looks like stops over the years are going down slightly. But there are two weird abberations: 2014 and 2016 seem _too_ low. We'll dive into this more in a bit!

### Back to data prep

Now let's get back to prepping our data. To get to our desired date range of the most recent full year (2015), we will 
1. Use the `filter()` function to specify the years we want, and 
2. Again use the assignment operator `<-` (it's like = in `R`) to create a new variable, `stops_2015`.

**tidyverse function: `filter()`**

* The `filter()` function is used to separate rows from the dataframe that
interest us from rows that do not. 
* In particular, `filter(DATA, CONDITION)`
returns `DATA` with all of the rows that satisfy `CONDITION` removed. 
* For
instance, we might want to only look at stops from 2015. To do this, we would type `stops %>% filter(yr == 2015)`, since we only want
rows from `stops` where the `yr` column is (i.e., `==`) `2015`. 
* We can also filter on multiple conditions, just separating each condition with a comma. So, for example, if we wanted all stops between 2011 and 2015, we would write `stops %>% filter(yr >= 2011, yr <=2015)`.

### Your turn

Create a new variable, `stops_2015` that is our stops dataframe filtered to just those that happened in the year (`yr`) 2015. 

In [0]:
# Use the filter function to get just stops from 2015


Just to be extra sure, let's check our date range in this new dataframe, `stops_2015`!

In [0]:
# What date range does stops_2015 cover?


### Another one!

Look back and the stop counts by year -- are there any years that look suspicious? (Remember, 2014 looked pretty low.) How could we use `filter()` and `range()` to investigate whether suspiciously low years have a full year of data?

In [0]:
# Try it out!


## Getting multiple answers at once!

To answer the problem above for a _different_ year, we'd have to filter to again to that individual year of data and test the date range of that year. Instead of doing this _for each_ year separately, let's do it all in one go! 

To do this, we basically want to apply the same command to each of the years, rather than filtering one at a time to each year. The key to doing this type of manipulation uses two new functions: `group_by` and `summarize`.

**tidyverse functions: `group_by()` and `summarize()`**

One thing that we often want to do with data is disaggregate it. That is, we
might want to take the data and break it down into smaller subpopulations. Then,
when we ask questions, we can ask about each piece---for instance, each
demographic group, each year, or each police district---instead of asking about the population as a whole.

The way to do this in `R` is with `group_by()` and `summarize()`. The standard way
to use `group_by()` is to call `group_by(DATA, COL_NAME)`, where 
* `DATA` is our dataframe and 
* `COL_NAME` is the name of a column. 
What `group_by()` then does is
take all the rows in the dataframe `DATA` and put them into different groups,
one for each different value in the column `COL_NAME`. So, for instance, if we
called `group_by(stops_w_yr, district)`, `R` would hand back to us the `stops_w_yr`
dataframe with all of its columns broken into different groups, one for each
police district. (Note: At this stage, the dataframe doesn't _look_ any different to the human eye, since the groupings are happening behind the scenes.)

Try it below!

In [0]:
stops_w_yr %>%
    group_by(district) %>%
    head()

The second step is to do something with those groups. That's what `summarize()`
does. The way `summarize()` works is to take a dataframe broken into groups by
`group_by()` and calculate a statistic for each group. The basic syntax is
`summarize(DATA, STAT = FUN(COL_NAME))`, where 
* `DATA` is some dataframe broken
up by `group_by()`, 
* `STAT` is some statistic we want to calculate, 
* `FUN` is the
function that calculates that statistic, and 
* `COL_NAME` is the name of the
column (or columns) used to calculate the statistic.

Let's put it all together with a few examples first.

In [0]:
stops_w_yr %>%
    group_by(district) %>%
    summarize(
        # The function n() just gives us the number of rows in each gropu
        n_stops = n(),
        n_arrests = sum(arrested),
        arrest_rate = n_arrests / n_stops
    )

In [0]:
stops_w_yr %>%
    group_by(district) %>%
    summarize(most_recent_stop = max(date))

## Exercise 2: Date ranges by year

**Using your new friends `group_by` and `summarize`, compute the min and max dates for each year.**

**(Hint: Using `range()` alone will give you an error, because it returns two values, which we can't shove into one column. Try using `min()` and `max()` to make two separate columns, or you can try doing some wizardly string magic to make a single column out of the `range()` output!)**

In [0]:
# YOUR CODE HERE


### Plotting

The method above gives us a sense of whether we are missing chunks of data at the beginning or end of a year, but what if we're missing data in the middle? It'd be nice to look at a plot of stops by day. 

Before doing that, though, let's build up some basic plotting skills in `R`.

The basics of plotting use a package called `ggplot2`. The `gg` stands for "grammar of graphics" -- basically, it's the language we use to describe plots. Each piece of the grammar (i.e., each element of the plot) is it's own function that can be layered onto our plot. So you start with a dataset with designated x- and y-axes, and then layer the elements you want: points, lines, bars, tweaking the x-axis, tweaking the title, etc. Each thing you'd want to do is a separate function. One difference between `ggplot` and what we've been doing before, is that the functions for plotting are strung together with `+` rather than with `%>%`.

We'll walk you through the first plot, and then you'll have ample opportunities to create your own plots throughout the afternoon!

In [0]:
stops_w_yr %>%
    count(yr) %>%
    # "aes()" stands for "aesthetic". 
    # Anything within `aes()` refers to information from our data (stops_w_yr)
    ggplot(aes(x = yr, y = n)) + 
    # geom_col just creates bar columns using the x and y coordinates specified
    geom_col()

### Exercise 3: Line graphs

**Instead of making a bar chart with`geom_col()`, try making a line graph, with points at each year. You can use the same intro as in our example, but instead of `geom_bar()`, try using `geom_point()` and `geom_line()`.** 

(Remember, you can always type `?geom_point` if you need help and want to read the manual! Or, you can check out [this cheatsheet](https://www.rstudio.com/wp-content/uploads/2015/03/ggplot2-cheatsheet.pdf)!)

In [0]:
# YOUR CODE HERE


### More plotting elements

A common thing to want to do is tweak the scales. The function to do this depends on what _type_ of data our axis represents (e.g., continuous, discrete, dates). In the case of the plot above, our y-axis looks a little funky. Let's use `scale_y_continuous()` to modify it.

In [0]:
stops_w_yr %>%
    count(yr) %>%
    # "aes()" stands for "aesthetic". 
    # Anything within `aes()` refers to information from our data (stops_w_yr)
    ggplot(aes(x = yr, y = n)) + 
    # geom_point adds points at the x and y coordinates specified
    geom_point() + 
    # geom_line draws lines between x and y coordinates specified
    geom_line() +
    # we can use the function below to control the y axis
    scale_y_continuous(
        name = "Number of stops", # Name your y-axis
        limits = c(0,150000), # Choose the min and max of the axis
        labels = scales::comma # Make the numbers pretty, with comma instead of sci notation
    )

### Exercise 4: More plotting exercises

  1.  **# If we want to look date-by-date, instead of year-by-year, we can just use `date` as our x-axis, and then we can call on the "histogram" function (which calculates the y-axis for us -- it's basically doing the samewould be the same as doing a `count(date)` and then a geom_col with x = date, y = n). Try Making a histogram of stops by date!**
  
  2.  **The benefit of `geom_histogram()` over the `count` and `geom_col` method, is that with `geom_histogram` you can control the granularity of your bars. Try adding `bins = 500` as an argument to `geom_histogram`. What changes? What happens when `bins = 10`? Play around with `bins` to see!**

In [0]:
# EXERCISE 4.1 and 4.2: YOUR CODE HERE


### Tricks with axes

In [0]:
# To focus in on the missing date range, we can use 
# `scale_x_date()` to add some breaks in our x-axis.
stops_w_yr %>%
    ggplot(aes(x = date)) +
    geom_histogram(bins = 500) +
    scale_x_date(date_breaks = "1 year") +
    theme(axis.text.x = element_text(angle = 45, hjust = 1))

### More elements: adding color

If we want to look at by-date _and_ by-gender trends, we can use _color_ to help distinguish between genders. When dealing with things like bars, cols, and histograms, which have _width_, we want to `fill` them with color. When we have elements like points and lines, we want to `color` them. 

Below, we simply add `fill = gender` to our `aes()`, and bam! Presto color!

In [0]:
# We can also look at the by-date, by-gender trends using "fill" 
# to distinguish between genders in our dataset
# NOTE: bars/cols/histograms use `fill = gender` to color by gender, 
#       points and lines use `color = gender`
stops_w_yr %>%
    ggplot(aes(x = date, fill = gender)) +
    geom_histogram(bins = 500) +
    scale_x_date(date_breaks = "1 year") +
    theme(axis.text.x = element_text(angle = 45, hjust = 1))

### Exercise 5: Putting it all together

  1. **Create a point and line plot that shows by-year stop counts colored by race.**
  
  2.  **Create another plot of your choosing. Think of a question about the data, or trends in the data, and see if you can answer that question with a plot! Alternatively, take one of the plots we've already made, and see if you can modify the axes/colors to make it better!**
  
  3.  **(Optional) If you want a quick way to make your plots interactive, check out the code below. Just save whatever plot you want to a variable (in this case `p`), and then run `plotly::ggplotly(p)`.**

In [0]:
# EXERCISE 5.1: YOUR CODE HERE


In [0]:
# EXERCISE 5.2: YOUR CODE HERE

In [0]:
# EXERCISE 5.3 (OPTIONAL)
install.packages("plotly")
library(plotly)
p <- # paste the code for your plot here!

plotly::ggplotly(p)

Nice! We'll get back to plotting in a bit. There are _endless_ tweaks and modifications and aesthetic optimizations you can fill your life with. We won't get into too many of those, but there are lots of resources online -- so once you get the basic structure or your plot, making it beautiful is well within reach!


## Back to calculating disparities

In that plotting aside we discovered 2014 and 2016 only have partial years. Let's return to our 2015 dataset, `stops_2015` and calculate how many stops were made of drivers of each race group.

## Exercise 6: Stops by race group

For this next exercise, let's compute the racial breakdown of traffic stops. To do this, we'll need two functions that we've already seen: `count()` and `mutate()`.

1. **Count how many stops per race group our `stops_2015` dataset has, saving your result to a new dataframe: `stops_by_race`.** 

2. **Describe in words what we'd need to do to find the proportion of stops that were of white drivers.** 

3. **To do the above computation for each race group, we can add additional column to `stops_by_race` using the `mutate()` function. Overwrite `stops_by_race`, adding a new column `p` with the proportion of stops that were made of drivers of each race group.**

4. **Discuss: What do these proportions mean? Are drivers of certain race groups being stopped more than others? What might we be missing to really start interpreting these values?**

In [0]:
# EXERCISE 6: YOUR CODE HERE


## Stop rates

In order to do this baseline comparison, we need to understand the racial
demographics in our SF population data. (Note: This is why we wanted just one full year: comparing the number of stops in a year to the population from that year.) The data as we've given it to you
has raw population numbers from 2015. To make it useful, we'll need to compute the
_proportion_ of SF residents in each demographic group. As before, we do this using the `mutate()` function.

### You try

* Take a look at `pop_2015`, then
* mutate `pop_2015`, adding a column `p` that shows us what proportion of the population is white, black, Hispanic, Asian, and other.

In [0]:
# Find the racial breakdown of SF's 2015 population


### Discuss

What do the population proportions tell you about the stop proportions we computed before?


### Adding rigor

We can tell a lot just by eyeballing these two sets of proportions. But let's be a bit more
rigorous about this. If we merge the two tables together, we can compute stop 
rates by race group (i.e., number of stops per person). 

**R function: `inner_join()`**

One way to put tables together is with the `inner_join()` function. We need to
input three things into this function: 
 1. our main table
 2. the second table we'd like to join with the first table, and
 3. instructions on how to join them. 

<img src="img/inner_join.png" alt="inner join" width="400" align="left"/>

In this case, the two tables we
want to merge are 
 1. the table of stops counted according to `race`, and
 2. the table of population by race: `pop_2015`. 
 
The instruction for combining the tables is 
 3. to merge rows that give information about the same race groups.

To implement 3., we give `inner_join()` the argument `by =
"race"`. This means that `inner_join()` will 
 * look at the first table---
i.e., the table stops counted by race---and go to the `race` column
in each row.
 * Then, `inner_join()` will take what it finds there---in this case,
`"asian/pacific islander"`, `"black"`, `"hispanic"`, `"other/unknown"`, and
`"white"`---and look in the second table, i.e., `pop_2015`, for all the
rows that contain the same information in `pop_2015`'s race column.
 * Finally,
it will add the second row at the end of the first to create a new row with
information from both. 

What we end up with is a dataframe with all of the
columns from _both_ tables.

The process is a little complicated, and we won't use it too much, so don't
worry if the abstract description doesn't make sense. To get a better
understanding of what's going on, exercise 5 will help guide you through merging the two tables described above,
being sure to include the `by = "race"` argument.


### You try

**Merge together `stops_by_race` and `pop_2015` by "race", using the `inner_join()` function. Name your result `stops_and_pop_by_race`.**

In [0]:
# YOUR CODE HERE


### Other joins

Note that `inner_join` isn't the only way to merge two tables together. While `inner_join` is like the intersection, `left_join` keeps all rows in the left dataframe, and merges in only rows from the right dataframe that match:
<img src="img/inner_ven.png" alt="inner join" width="250" align="left"/><img src="img/left_ven.png" alt="inner join" width="250" align="left"/>
So in table form, here's what left join does:
<img src="img/left_join.png" alt="inner join" width="400" align="left"/>

Another join that can sometimes be helpful is `anti_join`, which is the opposite of `inner join`. It keeps only the rows from the left dataframe that don't match to any rows in the left dataframe:
<img src="img/anti_join.png" alt="inner join" width="400" align="left"/>

These are good to know about (similar to SQL). For now, though, let's head back to our core question.

## Exercise 7: Computing stop rates by race group

1. **Take your new dataframe, `stops_and_pop_by_race`, and add a column called `stop_rate` that is simply the number of stops divided by the number of people.** (Hint: the `mutate()` function will be helpful!)
2. **Now we can divide the black (or Asian, or Hispanic, or "other") stop rate by the white stop rate to be able to make a quantitative statement about how much more often black drivers are stopped compared to white drivers, relative to their share of the city's population. Using `R` as a calculator, do this!**
3. **Discuss your results.**

In [0]:
# EXERCISE 7: YOUR CODE HERE


### Thought exercise: where stop rates fall short

While these baseline stats give us a sense that there are racial disparities in
policing practices in SF, they are not strong evidence of discrimination. The
argument against using stop rates (often called "benchmarking" or the "benchmark test") is that we haven't identified the correct
baseline to compare to. 
* Why isn't population the best thing to compare to (i.e., the best denominator of our stop rate)?
* What would the ideal denominator of our stop rate be?
* What other baselines (denominators) could we use? Are any of these ideal?

In [0]:
# Your thoughts here (or just discuss)


## Searches

Let's next consider how often drivers of different race groups were searched. Computing search rates is actually easier than stop rates because we don't need an external population benchmark.
We can use the stopped population as our baseline, defining search rate to be the proportion of stopped people who were subsequently searched. 

## Exercise 8: Search rates

1. **Compute search rates by race group.** (Hint: Think about what information you'd need to compute a search rate. The `n()` function might be helpful!)

2. **Discuss the search rate findings. Are some race groups searched more often than other race groups, relative to their share of stopped drivers?**

NOTE: Since we're not comparing to population numbers, we can return to using our full `stops_w_yr` dataset, with all years, because in this case it doesn't matter that we don't have full years for 2014 and 2016.

In [0]:
# YOUR CODE HERE


## Exercise 9: Tricks with booleans

Below is one possible solution to Exercise 6. The function `mean()` is defined over _numbers_, but `searched` is a column of booleans (TRUE/FALSE). Why does this solution work? Play around with treating booleans like numbers. What do you discover? (This behavior is called _coercion_ or _implicit casting_.)

In [0]:
stops_w_yr %>%
    group_by(race) %>%
    summarize(
        search_rate = mean(searched)
    )

## Thought exercise: where search rates fall short
* Do search rates have similar issues as we found with stop rates? Why or why not?
* What might "justifiably" lead search rates to differ by race group?

In [0]:
# Write your thoughts here (or just discuss)!


## Outcome test

To circumvent the benchmarking problem, it's common to turn to the search 
decision, rather than the stop decision. This is because we have a notion of
what a "successful" search is. The legal justification for performing a search
is probable cause that the driver possesses contraband. So a successful search
is one which uncovers contraband.

We thus turn to rates of successful searches. That is, what proportion of
searches, by race, were successful? This proportion is known as the contraband
recovery rate, or the "hit rate." If racial groups have different hit rates, it
can imply that racial groups are being subjected to different standards.

## Thought Exercise: Hit rate interpretation

As a caricatured example, suppose among white drivers who were searched, 
officers found contraband 99% of the time, while among black drivers who were
searched, officers found contraband only 1% of the time. 
* Is this police department's search policy discriminatory? 
* Why or why not?
* In general how can we use hit rates to understand whether a search policy is discriminatory?

In [0]:
# Write your thoughts here (or just discuss)!



Next let's investigate a non-caricatured case: real hit rates by race group in SF.

## Exercise 10: Hit rates

1. **Filter to drivers who were searched, and then compute the hit rate (rate of contraband recovery) by race group. Remember your `group_by()` and `summarize()` functions!**

2. **Discuss your findings.**

In [0]:
# YOUR CODE HERE


What if hit rates vary by police district? If the bar for stopping
people, irrespective of race, is lower in certain police districts, and black
individuals are more likely to live in neighborhoods in those districts, then
the observed disparities may not reflect bias.

Let's compute hit rates by race _and_ district. We can do this simply by adding multiple arguments to the `group_by()` function. Run the code below.

In [0]:
hit_rates <- 
  stops_w_yr %>% 
  filter(searched) %>% 
  group_by(race, district) %>% 
  summarize(hit_rate = mean(contraband_found))

hit_rates %>% nrow()

This is too many hit rates to compare in one table!

## Exercise 11: Visualization brainstorm

Sketch out using pen and paper (or just describe to a partner) how you might try to use visualizations to help us synthesize the 50 hit rates above. Start with the question we're trying to answer (Are hit rates for minority drivers lower than hit rates for white drivers?) -- and then think about what type of plot might best help you answer that question. See if you can come up with at least 3 different sketches!

## One way to visualize: scatterplots

One way to visualize the data is to make a scatterplot comparing white hit rates to each non-white race group. To do this, though, we need to reshape our data, so that each row allows us to compare white hit rate in a district, to each minority hit rate in that same district. 

The code below uses some tricky functions `spread` and `gather`. They're notoriously mind-bending, so the `tidyverse` team is actually developing new functions (`pivot_wider()` and `pivot_longer()`) to replace them soon. We'll walk you through this below, but don't get too hung up on it, since soon (like any day now) there will be more intuitive versions of these functions released.

In [0]:
# Reshape table to show hit rates of minorities vs white drivers
reshaped_hit_rates <-
  hit_rates %>% 
  # the column names (key) will be race, the values will be from hit rate
  spread(key = race, value = hit_rate, fill = 0) %>% 
  rename(white_hit_rate = white) %>% 
  # gather leaves us with two new columns: 
  # minority race (which contain the old colnames)
  # and minority hit rate (which contains the values in the old columns)
  gather(
      key = minority_race, value = minority_hit_rate, 
      # columns to gather:
      c(black, hispanic, `asian/pacific islander`, other)
  ) %>%
  arrange(district)

head(hit_rates %>% arrange(district))
head(reshaped_hit_rates)

Now we're ready to make a basic scatter plot! We'll walk you through it in the exercise below.

## Exercise 12: Scatterplot of racial disparities

1. **Warm-up. Before we plot, let's do some tidyverse magic. To make our plot nice in step 2, we'll need the range of hit rates. Create a variable `max_hit_rate` that extracts the largest hit rate value in `reshaped_hit_rates`. There are many many different ways to do this, so don't be afraid to play around!**

In [0]:
# YOUR CODE (EXERCISE 12, Q1)


  2.**Onward to plotting! First let's start simple. Filter `reshaped_hit_rates` to just `black` and `white` race groups (i.e., let's only consider rows where `minority_race` is "black"). Let's next create a scatterplot comparing black and white hit rates.** 

Consider the following questions/hints:
  * What do you want your x- and y-axes to be?
  * Which `geom_X` would you use to creat scatterplot points?
  * We want to compare apples to apples, so we want the x- and y-axis scales to cover the same range. To do this, we can use `scale_x_continuous()` and `scale_y_continuous()`. Both of these functions have an argument called `limits`. The basic usage is `scale_x_continuous(limits = c(A, B))`, where `A` is the lower limit of your range (we can just use 0), and `B` is the upper limit of your range (we can `max_hit_rate` from Q1 above). 
    Try adding and removing these two lines of code -- how does it change the plot? Why is it critical in this case?

In [0]:
# YOUR CODE (EXERCISE 12, Q2)


3.**Now, to incorporate all minority race groups in our dataset, let's return to the unfiltered `reshaped_hit_rates`. Instead, let's create a panel for each race-pair we're comparing. We can do this simply by adding `facet_wrap(facets = vars(minority_race))` to the previous code. Try it! What do you see?**

In [0]:
# YOUR CODE (EXERCISE 12, Q3)


4.**Starting from your code in part 3 of this exercise, make the following changes to your plot:**
  * **Use `labs()` to relabel your axes.** 
  * **Use `geom_abline()` to make a dashed trendline that marks the "line of fairness", i.e., the line of equal hit rates by race. (Hint: _slope_ and _intercept_ are both arguments of `geom_abline()`)**

In [0]:
# YOUR CODE (EXERCISE 12, Q4)


Below are a few slight tweaks to make the plot a bit more readable. Let's go through these additions together to make sure they make sense. Below, we're going to:

* Size the points by number of searches
* Use percentages rather than proportions for the axes

In [0]:
# Get corresponding number of searches (to size points).
# Again, for each district we want to know the number of white+black searches
# and white+Hispanic searches. This requires the same spreading and gathering
# as our previous data-munging.
search_counts <-
  stops_w_yr %>% 
  filter(searched) %>%  
  count(district, race) %>% 
  spread(race, n, fill = 0) %>% 
  rename(num_white_searches = white) %>% 
  gather(
      minority_race, num_minority_searches, 
      c(black, hispanic, `asian/pacific islander`, other)
  ) %>% 
  mutate(num_searches = num_minority_searches + num_white_searches) %>% 
  select(district, minority_race, num_searches)

# Now let's plot!
reshaped_hit_rates %>% 
  left_join(
    search_counts, 
    by = c("district", "minority_race")
  ) %>% 
  ggplot(aes(
    x = white_hit_rate,
    y = minority_hit_rate
  )) +
  geom_point(aes(size = num_searches), pch = 21) +
  geom_abline(slope = 1, intercept = 0, linetype = "dashed") +
  scale_x_continuous("White hit rate", 
    limits = c(0, max_hit_rate + 0.01),
    labels = scales::percent_format(accuracy = 1),
    # forces axis to start _right_ at zero, no gap
    expand = c(0,0)
  ) +
  scale_y_continuous("Minority hit rate", 
    limits = c(0, max_hit_rate + 0.01),
    labels = scales::percent_format(accuracy = 1),
    # forces axis to start _right_ at zero, no gap
    expand = c(0,0)
  ) +
  coord_fixed() +
  facet_wrap(vars(minority_race))

## Exercise 13: Plot interpretation

**Explain what you see above. What does each point represent? What does the dotted line represent? What do these plots tell us about discrimination in search practices?**

# Modeling

Plotting allowed us to synthesize fifty hit rates into an understanding of racial disparities in SF's search policy. Our plot provides us with evidence that these searches are discriminatory against black and Hispanic drivers. 

What if we want to _quantify_ this disparity? Our plot just gives us intuition. Similar to how we were able to compute the ratio between black hit rates and white hit rates for the aggregate hit rate data (citywide), we might want to know what those ratios would be in each district. Instead of separately computing the hit rate for each race/district, we can instead use modeling to put numbers on these disparities. We can use *regression*: computing how likely an officer was to find contraband given the drivers' race and district.

We're not going to teach the statistics behind regression right now. If you already know about regression, we're here to teach you how to run a regression in `R`. If you've never heard of regression before, maybe this short soiree into modeling will inspire you to learn! 

Our _outcome_ or _response_ variable, `contraband_found`, is a boolean (TRUE/FALSE, or 1/0) -- also known as a "binomial" variable. Recall that when we have an binomial outcome like this, we use logistic regression.

Below, we create a dataframe with just the searches, and we convert all categorical variables to factors. A "factor" in `R` is has explicit levels -- it's telling the model not to derive a coefficient for each value within `race` and `district` (rather than treating them as continuous variables, with a single coefficient).

For the model itself, we use `glm()`, which stands for "generalized linear model". We simply give `glm()` a formula: `response ~ covariate_1 + covariate_2 + ... + covariate_n`, the data, and then we specify the type of model we want to fit using the `family` argument. Logistic regression is "binomial" family.

In [0]:
library(broom) #library for turning messy r output to tidy dataframes

searched_df <- stops_w_yr %>% 
    filter(searched) %>%
    mutate(
        race = factor(race),
        district = factor(district)
    )

mod <- glm(
    contraband_found ~ race + district, 
    data = searched_df, 
    family = 'binomial'
)

tidy(mod)

Notice that there is no term for "asian/pacific islander" or "district A" in the above table.  This is because they were set as reference levels, which happens whenever you use a categorical variable in a regression.  We want to set white as the reference level for race so that we can read the coefficients for other races as comparisons to white hit rates. We can do this using the `relevel` function. 

In [0]:
searched_df <- searched_df %>%
    mutate(race = relevel(race, ref = 'white'))

mod <- glm(
    contraband_found ~ race + district, 
    data = searched_df, 
    family = 'binomial'
)
tidy(mod)

Now let's summarize our results by pulling out only the information we care about.

In [0]:
tidy(mod) %>%
    filter(str_detect(term, "race")) %>%
    mutate(
        race = str_remove_all(term, "race"),
        lower_ci = round(estimate - 1.96*std.error, 3),
        upper_ci = round(estimate + 1.96*std.error, 3),
        likelihood_recovery_v_white = exp(estimate)
    ) %>%
    select(
        race, 
        estimate, 
        lower_ci, 
        upper_ci, 
        likelihood_recovery_v_white
    )

A _negative_ coefficient estimate tells us that searches of that race group are _less_ likely to recovery contraband than searches of white drivers in the same district.

In line with our plots, we see that searches of both black and Hispanic drivers recover significantly less contraband than searches of white drivers. By exponentiating the point estimates, we can acquire values than allow us to make quantitative statements: Searches of black and Hispanic drivers recover contraband only 35% as often as searches of white drivers.

### Exercise 14

1. **Fit a model to predict how likely a person is to be searched, given their race, district, gender, age, and reason for stop. Name your model `search_mod`.** Hint: Remember to turn _categorical_ variables into "factors", and to relevel your race variable to be able to answer part 2. 

2. **Use your model to answer: How much more likely to be searched are black and Hispanic drivers compared to similarly situated white drivers? (i.e., compared to white drivers of the same age and gender, stopped in the same district for the same reason)**

In [0]:
# EXERCISE 14.1: YOUR CODE HERE


In [0]:
# EXERCISE 14.2: YOUR CODE HERE


### Model prediction

Often times you'll want to train a model in order to use it to make predictions. Let's use your model from Exercise 14, `search_mod`, to help predict the likelihood of being searched for a white female, age 27, living in district E, who was stopped for a moving violation.

Here's how we'd do that:

In [0]:
person <- tibble(
    race = c('white'),
    gender = c('female'),
    age = c(27),
    district = c('E'),
    reason_for_stop = c('Moving violation')
)
predict(search_mod, person, type = 'response') %>% round(3)

**Statistical note:** We need `type = 'response'` so that R knows we want it to output probabilities.  Otherwise, it will output log odds, which is the scale that logistic regression uses.  

### Exercise 15: Many predictions

Instead of just passing one person to `predict()`, we can pass in lots of different people! 

1. Create a dataframe (or "tibble") called `people`, with 10 rows and two columns. As with `person` our columns will be "race", "gender", "age", "district", and "reason_for_stop". 
  * Let's make each of them in a different district (there are 10 districts in our dataset).
  * Half male, half female.
  * Your choose their races.
  * You choose their ages!
  * You choose their reasons for being stopped. (Make sure that you're choosing reasons from the ones listed in our dataset -- these are categorical.)
  
  Hint: the function `rep()` might be helpful!

2. Now use `predict` to calculate how likely your people is to be searched in each district in SF.

In [0]:
# YOUR CODE HERE